In [0]:
dbutils.widgets.text('bronze_catalog_name','')
dbutils.widgets.text('bronze_schema_name','')
dbutils.widgets.text('bronze_table_name','')
dbutils.widgets.text('silver_catalog_name','')
dbutils.widgets.text('silver_schema_name','')
dbutils.widgets.text('silver_table_name','')

In [0]:
bronze_catalog_name = dbutils.widgets.get('bronze_catalog_name')
bronze_schema_name = dbutils.widgets.get('bronze_schema_name')
bronze_table_name = dbutils.widgets.get('bronze_table_name')
silver_catalog_name = dbutils.widgets.get('silver_catalog_name')
silver_schema_name = dbutils.widgets.get('silver_schema_name')
silver_table_name = dbutils.widgets.get('silver_table_name')
if bronze_catalog_name == '':
  raise Exception('Please provide a bronze catalog name')
if bronze_schema_name == '':
  raise Exception('Please provide a bronze schema name')
if bronze_table_name == '':
  raise Exception('Please provide a bronze table name')
if silver_catalog_name == '':
  raise Exception('Please provide a silver catalog name')
if silver_schema_name == '':
  raise Exception('Please provide a silver schema name')
if silver_table_name == '':
  raise Exception('Please provide a silver table name')

In [0]:
from pyspark.sql import functions as F

df = spark.table(f'{bronze_catalog_name}.{bronze_schema_name}.{bronze_table_name}')

# Valida que el campo id sea unico
id_count = df.select('id').distinct().count()
total_count = df.count()
if id_count != total_count:
    raise Exception("La columna 'id' no es única en el dataframe.")

# Validar que la columna 'email' tenga un correo válido
invalid_email_count = df.filter(~F.col('email').rlike(r'^[A-Za-z0-9._%+-]+@[A-Za-z0-9.-]+\.[A-Za-z]{2,}$')).count()
if invalid_email_count > 0:
    raise Exception("La columna 'email' contiene correos no válidos.")

# Validar que la columna 'telefono' no tenga nulos
null_telefono_count = df.filter(F.col('telefono').isNull()).count()
if null_telefono_count > 0:
    raise Exception("La columna 'telefono' contiene valores nulos.")

df.write.mode("overwrite").format("delta").saveAsTable(f'{silver_catalog_name}.{silver_schema_name}.{silver_table_name}')